In [1]:
import pandas as pd
import json
import sys
import pickle
sys.path.append('db')
from db.select_training_data import get_training_data
from preprocessing import Preprocessing
from sklearn.model_selection import train_test_split

In [2]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

In [3]:
df = get_training_data()
pp = Preprocessing(df)
data = pp.fit()
data

/mnt/c/Users/victo/OneDrive/Desktop/League/League_ARAM_prediction_model-main/db/select_training_data.py:42: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(select_query, connection)


,game_result,level_p1,champ_difficulty_p1,total_poke1,hard_cc_champ1,level_p2,champ_difficulty_p2,total_poke2,hard_cc_champ2,level_p3,...,Sejuani,Qiyana,Udyr,RekSai,num_assassin,num_fighters,num_mages,num_marksmen,num_supports,num_tanks
0,W,247,10,2,2,801,10,0,1,47,...,0,0,0,0,1,2,3,0,1,1
1,L,739,3,2,1,177,8,1,1,112,...,0,0,0,0,2,2,1,1,1,1
2,W,674,4,0,2,104,6,2,1,106,...,0,0,0,0,1,2,2,1,1,2
3,L,177,7,0,0,465,6,1,0,34,...,0,0,0,0,1,0,2,2,3,0
4,W,46,2,2,1,564,6,2,1,470,...,0,0,0,0,2,2,1,1,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36143,L,119,4,1,1,37,9,2,1,317,...,0,0,0,0,2,2,3,0,1,0
36144,W,53,3,0,1,392,1,3,0,400,...,0,0,0,0,0,3,1,1,0,3
36145,L,81,3,2,3,320,4,1,1,518,...,0,0,0,0,0,1,3,1,1,2
36146,W,718,5,1,0,255,7,0,1,118,...,0,0,0,0,3,4,1,0,2,0


In [4]:
y = data['game_result']
X = data.drop(['game_result'], axis=1)

y = y.map({'L': 0, 'W': 1})

In [5]:
X.drop(['level_p1', 'level_p2', 'level_p3', 'level_p4', 'level_p5'], axis=1, inplace=True)

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=0)

In [13]:
X_train.columns

Index(['champ_difficulty_p1', 'total_poke1', 'hard_cc_champ1',
       'champ_difficulty_p2', 'total_poke2', 'hard_cc_champ2',
       'champ_difficulty_p3', 'total_poke3', 'hard_cc_champ3',
       'champ_difficulty_p4',
       ...
       'Sejuani', 'Qiyana', 'Udyr', 'RekSai', 'num_assassin', 'num_fighters',
       'num_mages', 'num_marksmen', 'num_supports', 'num_tanks'],
      dtype='object', length=187)

In [15]:
param_grid={'C': [0.01, 0.1, 1, 10], 
            'penalty': [None, 'l1', 'l2', 'elasticnet'],
            'max_iter': [100, 1000, 10000]}

In [17]:
clf_log = GridSearchCV(estimator=LogisticRegression(random_state=0, n_jobs=-1), param_grid=param_grid, n_jobs=-1, cv=5)

In [18]:
clf_log.fit(X_train, y_train)

/mnt/c/Users/victo/OneDrive/Desktop/League/League_ARAM_prediction_model-main/league_app/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1193: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/mnt/c/Users/victo/OneDrive/Desktop/League/League_ARAM_prediction_model-main/league_app/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1193: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/mnt/c/Users/victo/OneDrive/Desktop/League/League_ARAM_prediction_model-main/league_app/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1193: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/mnt/c/Users/victo/OneDrive/Desktop/League/League_ARAM_prediction_model-main/league_app/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1193: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(


GridSearchCV(cv=5, estimator=LogisticRegression(n_jobs=-1, random_state=0),
             n_jobs=-1,
             param_grid={'C': [0.01, 0.1, 1, 10],
                         'max_iter': [100, 1000, 10000],
                         'penalty': [None, 'l1', 'l2', 'elasticnet']})

In [19]:
clf_log.predict_proba(X_test)

array([[0.49516359, 0.50483641],
       [0.43084881, 0.56915119],
       [0.47325173, 0.52674827],
       ...,
       [0.40987737, 0.59012263],
       [0.50687156, 0.49312844],
       [0.43153069, 0.56846931]])

In [22]:
accuracy_score(y_test, clf_log.predict(X_test))

0.512863070539419

In [8]:
param_rf={'n_estimators': [100, 1000, 2500], 
            'max_depth': [None, 3, 10],
            'min_samples_split': [1, 5, 10]}

In [9]:
clf_rf = RandomizedSearchCV(estimator=RandomForestClassifier(random_state=0, n_jobs=-1), n_iter=25, param_distributions=param_rf, n_jobs=-1, cv=5)

In [10]:
clf_rf.fit(X_train, y_train)

/mnt/c/Users/victo/OneDrive/Desktop/League/League_ARAM_prediction_model-main/league_app/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/mnt/c/Users/victo/OneDrive/Desktop/League/League_ARAM_prediction_model-main/league_app/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
45 fits failed out of a total of 125.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
45 fits failed with the following error:
Traceback (most recent call last):
  File "/mnt/c/Users/victo/OneDrive/Desktop/League/League_ARAM_prediction_model-

RandomizedSearchCV(cv=5,
                   estimator=RandomForestClassifier(n_jobs=-1, random_state=0),
                   n_iter=25, n_jobs=-1,
                   param_distributions={'max_depth': [None, 3, 10],
                                        'min_samples_split': [1, 5, 10],
                                        'n_estimators': [100, 1000, 2500]})

In [11]:
accuracy_score(y_test, clf_rf.predict(X_test))

0.5427385892116182

In [18]:
param_xgb = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'max_depth': [3, 6, 9]
        }

In [30]:
xgb = XGBClassifier(eta=0.2, objective='binary:logistic')

In [36]:
random_search = RandomizedSearchCV(xgb, param_distributions=param_xgb, scoring='accuracy', n_iter=25, n_jobs=-1, cv=3, random_state=0 )

In [37]:
random_search.fit(X_train, y_train)

RandomizedSearchCV(cv=3,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, device=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False, eta=0.2,
                                           eval_metric=None, feature_types=None,
                                           gamma=None, grow_policy=None,
                                           importance_type=None,
                                           interaction_constraints=None,
                                           learn...
                                           max_delta_step=None, max_depth=None,
                                           max_leaves=None,
                                           min_child_weight=None, missing=nan,
                                           monotone_constraints=None,
                                           multi_strategy=None,
                                           n_estimators=None, n_jobs=None,
                                           num_parallel_tree=None, ...),
                   n_iter=25, n_jobs=-1,
                   param_distributions={'gamma': [0.5, 1, 2, 5],
                                        'max_depth': [3, 6, 9],
                                        'min_child_weight': [1, 5, 10],
                                        'subsample': [0.6, 0.8, 1.0]},
                   random_state=0, scoring='accuracy')

In [38]:
accuracy_score(y_test, random_search.predict(X_test))

0.5327800829875519

In [12]:
with open('model.pkl', 'wb') as file:  
    pickle.dump(clf_rf, file)